In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ============================================================
# Physics-Informed Neural Network (PINN) for 1D Burgers' Equation in (x, t)
#
# PDE (here, with a modified viscosity term):
#   u_t + u * u_x - (nu / pi) * u_xx = 0   on   x ∈ [-1, 1], t ∈ [0, 1]
#
# Initial condition (IC):
#   u(x, 0) = -sin(pi * x)
#
# Boundary conditions (BC, here Dirichlet and homogeneous):
#   u(-1, t) = 0,  u(1, t) = 0
#
# The PINN learns u(x, t) by minimizing:
#   - IC loss  (fit u(x,0) to initial_condition(x))
#   - BC loss  (fit u at x=-1,1 to boundary_condition)
#   - PDE loss (residual of the Burgers equation at collocation points)
# ============================================================


# ============================================================
# 1. Define the neural network model (PyTorch)
# ============================================================
class PINN(nn.Module):
    """
    Simple fully-connected neural network (MLP) that takes
    (x, t) as input and outputs scalar u(x,t).

    Input dimension:  2 (spatial coordinate x, time coordinate t)
    Output dimension: 1 (solution u(x,t))

    Architecture:
      (x,t) -> Linear(2 -> 20) -> Tanh ->
               Linear(20->20) -> Tanh ->
               Linear(20->20) -> Tanh ->
               Linear(20->20) -> Tanh ->
               Linear(20->1)  -> u
    """
    def __init__(self):
        super(PINN, self).__init__()

        # Sequential container for a stack of fully-connected layers + activations.
        # This defines the nonlinear mapping (x,t) -> u(x,t).
        self.hidden = nn.Sequential(
            nn.Linear(2, 20),  # First layer: 2 input features (x, t) -> 20 hidden units
            nn.Tanh(),         # Tanh nonlinearity
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1)   # Last layer: 20 -> 1 output (scalar u)
        )

        # --- Optional: inverse-problem parameter for viscosity ν ---
        # If you want to learn ν from data (inverse problem), you can uncomment:
        #
        #   self.nu = nn.Parameter(torch.rand(1, dtype=torch.float32))
        #
        # This registers ν as a trainable parameter of the network.
        # In the PDE residual, you would then use model.nu instead of a fixed nu.


    def forward(self, x, t):
        """
        Forward pass of the network.

        Inputs:
          x : tensor of shape (N, 1), spatial coordinates
          t : tensor of shape (N, 1), temporal coordinates

        Steps:
          1) Concatenate x and t along feature dimension -> shape (N, 2)
          2) Pass through the hidden MLP to predict u(x,t).
        """

        # Concatenate x and t into a single (N,2) input tensor:
        # dim=1 means we concatenate columns; each row is [x_i, t_i].
        inputs = torch.cat([x, t], dim=1)

        # Apply the fully-connected network to obtain u predictions
        u = self.hidden(inputs)

        # Output shape is (N,1)
        return u


# ============================================================
# 2. PDE residual: Burgers' equation
# ============================================================
def pde_residual(x, t, model, nu=0.01):
    """
    Compute the residual of the Burgers' PDE at given collocation points (x,t).

    PDE (here written as residual = 0):
        u_t + u * u_x - (nu / pi) * u_xx = 0

    So the residual is:
        r(x,t) = u_t + u * u_x - (nu / pi) * u_xx

    Inputs:
      x     : tensor of shape (N,1), spatial collocation points
      t     : tensor of shape (N,1), temporal collocation points
      model : PINN instance that approximates u(x,t)
      nu    : viscosity-like coefficient (scalar, float)

    Output:
      residual : tensor of shape (N,1) representing PDE residual at each point
    """

    # IMPORTANT:
    # We must tell PyTorch to track gradients of x and t w.r.t. the computational graph,
    # so that autograd can compute derivatives u_t, u_x, u_xx.
    x.requires_grad_(True)
    t.requires_grad_(True)

    # Forward pass: get network prediction u(x,t)
    u = model(x, t)  # shape (N,1)

    # First derivative w.r.t time: u_t = du/dt
    #   autograd.grad arguments:
    #     outputs=u
    #     inputs=t
    #     grad_outputs=torch.ones_like(u)  (vector-Jacobian product, picks elementwise derivative)
    #   create_graph=True -> keep graph for higher-order derivatives if needed.
    u_t = torch.autograd.grad(
        u, t,
        grad_outputs=torch.ones_like(u),
        create_graph=True)[0]  # shape (N,1)

    # First derivative w.r.t space: u_x = du/dx
    u_x = torch.autograd.grad(
        u, x,
        grad_outputs=torch.ones_like(u),
        create_graph=True)[0]  # shape (N,1)

    # Second derivative w.r.t space: u_xx = d^2u/dx^2
    #   We differentiate u_x w.r.t x again.
    u_xx = torch.autograd.grad(
        u_x, x,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True
    )[0]  # shape (N,1)

    # PDE residual:
    #   r = u_t + u * u_x - (nu/pi) * u_xx
    # The PINN will try to make r(x,t) ≈ 0 at all collocation points.
    residual = u_t + u * u_x - nu / (np.pi) * u_xx

    return residual  # (N,1)


# ============================================================
# 3. Initial and boundary conditions
# ============================================================
def initial_condition(x):
    """
    Initial condition at t = 0.

    IC: u(x, 0) = -sin(pi * x)

    Input:
      x: tensor of shape (N,1)
    Output:
      u_IC: tensor of shape (N,1)
    """
    return -torch.sin(np.pi * x)


def boundary_condition(x, t):
    """
    Boundary condition u(x,t) on x = -1 and x = 1.

    Here: homogeneous Dirichlet BC (u = 0 on the boundary).
    We ignore x and just return zero with the same shape as t.

    Inputs:
      x: boundary x-values (unused)
      t: times on the boundary

    Output:
      tensor of zeros of same shape as t
    """
    return torch.zeros_like(t)


# ============================================================
# 4. Create training data: interior collocation grid
# ============================================================
# Spatial coordinates for initial condition (and also used for PDE collocation)
x = torch.linspace(-1, 1, 200).view(-1, 1)  # shape (200,1)
# Temporal coordinates for boundary/PDE collocation
t = torch.linspace(0, 1, 100).view(-1, 1)   # shape (100,1)

# Build a grid of (x,t) collocation points for PDE residual
# meshgrid with indexing='xy':
#   x_train[i,j] = x_i
#   t_train[i,j] = t_j
x_train, t_train = torch.meshgrid(
    x.squeeze(), t.squeeze(), indexing='xy'
)
# Flatten the 2D grids into 1D lists of coordinates:
#   shape (200*100, 1) = (20000,1)
x_train = x_train.reshape(-1, 1)
t_train = t_train.reshape(-1, 1)

# --- Optional: Inverse problem data (viscosity estimation) ---
# If you have reference solution data (x_ref, t_ref, exact), you can create:
#
# x_data = torch.tensor(x_ref.flatten(), dtype=torch.float32).view(-1, 1)
# t_data = torch.tensor(t_ref.flatten(), dtype=torch.float32).view(-1, 1)
# u_data = torch.tensor(exact.flatten(), dtype=torch.float32).view(-1, 1)
#
# And then add a data-fitting loss: MSE between model(x_data,t_data) and u_data.
# --------------------------------------------------------------


# ============================================================
# 5. Instantiate the model, optimizer
# ============================================================
model = PINN()  # by default on CPU; can .to(device) if GPU is used
optimizer = optim.Adam(model.parameters(), lr=0.001)


# ============================================================
# 6. Training loop
# ============================================================
num_epochs = 12000
for epoch in range(num_epochs):
    model.train()

    # ========================================================
    # NOTE for inverse problem:
    #   For viscosity estimation with data, you would typically
    #   use (x_data, t_data) and a data loss term.
    #   The PDE residual often uses (x_train, t_train) as here.
    #   This comment says that in that setting, some terms may
    #   use x_train, t_train instead of x, t.
    # ========================================================

    # -----------------------
    # (a) Initial condition loss
    # -----------------------
    # Compute model prediction at t=0:
    # Collocation points for initial condition: u_pred(x, 0) ≈ u(x, 0)
    u_pred = model(x, torch.zeros_like(x))  # input t=0 for all x

    # True IC values from the analytical initial condition function
    u_true = initial_condition(x)

    # Mean squared error of initial condition mismatch
    loss_ic = torch.mean((u_pred - u_true) ** 2)

    # -----------------------
    # (b) Boundary condition loss: Model predictions at boundaries:
    # -----------------------
    # Collocation points for boundary conditions
    # 1) Left boundary x = -1, for all t
    x_left = torch.full_like(t, -1.0)  # shape (100,1) all entries = -1
    # u(-1,t)
    u_pred_left = model(x_left, t)   # shape (100,1)

    # 2) Right boundary x =  1, for all t
    x_right = torch.full_like(t, 1.0)  # shape (100,1) all entries = 1
    # u(1,t)
    u_pred_right = model(x_right, t) # shape (100,1)

    # Target boundary values (here zero, homogeneous Dirichlet)
    u_bc_left = boundary_condition(x_left, t)
    u_bc_right = boundary_condition(x_right, t)

    # MSE on both boundaries
    loss_bc = torch.mean((u_pred_left - u_bc_left) ** 2) + \
              torch.mean((u_pred_right - u_bc_right) ** 2)

    # -----------------------
    # (c) PDE residual loss
    # -----------------------
    # Compute PDE residual at interior collocation points (x_train, t_train)
    residual = pde_residual(x_train, t_train, model)
    # Mean squared residual (PINN forces this toward zero)
    loss_pde = torch.mean(residual ** 2)

    # -----------------------
    # (d) Total loss
    # -----------------------
    # Here all three components are equally weighted; in practice, you
    # might tune weights: loss = w_ic*loss_ic + w_bc*loss_bc + w_pde*loss_pde.
    loss = loss_ic + loss_bc + loss_pde

    # -----------------------
    # (e) Backpropagation and optimizer step
    # -----------------------
    optimizer.zero_grad()  # clear previous gradients
    loss.backward()        # compute gradients d(loss)/d(theta)
    optimizer.step()       # update model parameters

    # Simple logging every 500 epochs
    if (epoch) % 500 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')



In [ ]:
# ============================================================
# 7. Evaluation: predict u(x,t) on a test grid for visualization
# ============================================================
# Create a regular grid in (x,t) for plotting
x_test = torch.linspace(-1, 1, 100).view(-1, 1)  # (100,1)
t_test = torch.linspace(0, 1, 100).view(-1, 1)   # (100,1)

# meshgrid for all combinations of (x_i, t_j)
x_test, t_test = torch.meshgrid(
    x_test.squeeze(), t_test.squeeze(), indexing='xy'
)  # x_test, t_test: shape (100,100)

# Flatten to shape (10000,1) so that each row is one (x,t) pair
x_test = x_test.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)

model.eval()
with torch.no_grad():
    # Predict u(x,t) at these grid points
    # Note: .cpu().numpy() is safer if later plotting with matplotlib
    u_pred = model(x_test, t_test).cpu().numpy()  # shape (10000,1)

# Reshape back to 2D fields for contour plotting:
#   x_test_grid[i,j], t_test_grid[i,j], u_pred_grid[i,j]
x_test = x_test.numpy().reshape(100, 100)
t_test = t_test.numpy().reshape(100, 100)
u_pred = u_pred.reshape(100, 100)


### Reference dataset
## Load the reference solution data
data = np.load("Burgers.npz")
t_ref, x_ref, exact = data["t"], data["x"], data["usol"].T

# Reshape x_ref and t_ref to match the shape of exact
x_ref, t_ref = np.meshgrid(x_ref, t_ref)



### Plotting: a comparison between reference data and PINN result
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Contour plot: PINN solution
contour1 = ax1.contourf(x_test, t_test, u_pred, levels=250, cmap='jet')
fig.colorbar(contour1, ax=ax1)
ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_title('Solution of Burgers\' Equation using PINNs (CPU)')

# Contour plot: reference solution
contour2 = ax2.contourf(x_ref, t_ref, exact, levels=250, cmap='jet')
fig.colorbar(contour2, ax=ax2)
ax2.set_xlabel('x')
ax2.set_ylabel('t')
ax2.set_title('Reference Solution of Burgers\' Equation')

# Set the same limits for both plots
ax1.set_xlim([x_ref.min(), x_ref.max()])
ax1.set_ylim([t_ref.min(), t_ref.max()])
ax2.set_xlim([x_ref.min(), x_ref.max()])
ax2.set_ylim([t_ref.min(), t_ref.max()])

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Plot reference vs PINN solution at selected time slices
# ============================================================

# Sample time instances at which we want to inspect u(x, t).
# Each value in this list corresponds to a horizontal "slice" in the (x, t) domain.
time_slices = [0.2, 0.4, 0.6, 0.8]  # You can add more time values here if needed.

# Total number of time slices we want to visualize.
num_plots = len(time_slices)

# We fix the number of columns for the subplot layout.
# Here, we choose 2 columns so that subplots are arranged in at most 2 per row.
cols = 2

# Compute how many rows we need in the subplot grid.
# Explanation:
#   - num_plots // cols : the number of full rows that can be filled completely
#   - num_plots % cols  : 1 if there is a partially filled row, 0 otherwise
# So if we have an odd number of plots, we add one extra row.
rows = (num_plots // cols) + (num_plots % cols)

# Create a figure with a grid of subplots having shape (rows, cols).
# figsize is scaled with 'rows' so the height grows with the number of rows.
fig, axs = plt.subplots(rows, cols, figsize=(12, 5 * rows))

# 'axs' is a 2D array of Axes when rows * cols > 1.
# Flatten it to a 1D array so we can index subplots as axs[0], axs[1], ...
axs = axs.flatten()

# ------------------------------------------------------------
# Loop over the desired time slices and plot each in a subplot
# ------------------------------------------------------------
for i, t_val in enumerate(time_slices):
    # --------------------------------------------------------
    # 1) Extract the reference solution at time t ~ t_val
    # --------------------------------------------------------

    # 't_ref' is assumed to be an array of reference time values with shape (Nt, 1).
    # We find the index of the time in t_ref that is closest to the desired t_val.
    # This allows us to approximate the reference solution at t = t_val
    # even if t_val is not exactly one of the t_ref grid points.
    idx_ref = np.argmin(np.abs(t_ref[:, 0] - t_val))

    # 'exact' is assumed to have shape (Nt, Nx), where:
    #   Nt : number of time steps in the reference solution
    #   Nx : number of spatial grid points.
    # Selecting 'exact[idx_ref, :]' gives the spatial profile u(x, t_ref[idx_ref])
    # at the chosen time slice closest to t_val.
    u_ref_slice = exact[idx_ref, :]

    # --------------------------------------------------------
    # 2) Compute the PINN prediction at the same time t_val
    # --------------------------------------------------------

    # We create a spatial grid x_slice over [-1, 1], with 100 points.
    # Shape: (100, 1)
    x_slice = np.linspace(-1, 1, 100).reshape(-1, 1)

    # For each x in x_slice, we want to evaluate u(x, t_val).
    # So we build t_slice by repeating t_val for all 100 x-points.
    # Shape: (100, 1)
    t_slice = t_val * np.ones((100, 1))

    # Switch the model into evaluation mode.
    # This is important if the model contains layers like Dropout or BatchNorm.
    model.eval()

    # Disable gradient computation, since we are only doing inference (no backprop).
    with torch.no_grad():
        # Convert x_slice and t_slice from NumPy arrays to PyTorch tensors
        # with dtype float32. Shape remains (100, 1) for both.
        x_tensor = torch.tensor(x_slice, dtype=torch.float32)
        t_tensor = torch.tensor(t_slice, dtype=torch.float32)

        # Forward pass through the PINN model:
        #   model(x_tensor, t_tensor) -> predicted u(x, t_val)
        # The output 'u_pinn_slice' is a tensor of shape (100, 1).
        u_pinn_slice = model(x_tensor, t_tensor).numpy()  # convert to NumPy for plotting

    # --------------------------------------------------------
    # 3) Plot reference vs PINN solution in subplot axs[i]
    # --------------------------------------------------------

    # x_ref is assumed to contain the spatial grid used by the reference solution.
    # For example, x_ref might have shape (1, Nx) or (Nt, Nx). Here we read the first row.
    # u_ref_slice is the reference solution at t ~ t_val, with shape (Nx,).
    axs[i].plot(x_ref[0, :], u_ref_slice, 'r-', label='Reference Solution')

    # Plot the PINN prediction at the same (or very similar) spatial points.
    # x_slice is shape (100, 1), u_pinn_slice is shape (100, 1).
    axs[i].plot(x_slice, u_pinn_slice, 'b--', label='PINN Solution')

    # Label the axes for the current subplot.
    axs[i].set_xlabel('x')   # horizontal axis: spatial coordinate
    axs[i].set_ylabel('u')   # vertical axis: solution value u(x, t)

    # Title indicates which time slice this subplot corresponds to.
    axs[i].set_title(f'Solution at t = {t_val}')

    # Add a legend to distinguish reference vs PINN curves.
    axs[i].legend()

# ------------------------------------------------------------
# 4) Remove any unused subplot axes (when rows*cols > num_plots)
# ------------------------------------------------------------

# If rows*cols > num_plots, we have extra axes that are not used.
# For example, with 3 plots and a 2x2 grid, axs has length 4 but we only used indices 0,1,2.
# The following loop removes the unused axes from the figure to avoid empty subplots.
for j in range(num_plots, len(axs)):
    fig.delaxes(axs[j])

# ------------------------------------------------------------
# 5) Final layout tweaks and display
# ------------------------------------------------------------

# Automatically adjust spacing between subplots so that labels, titles, and legends
# do not overlap.
plt.tight_layout()

# Render the complete figure with all subp
::contentReference[oaicite:0]{index=0}


In [ ]:
# ============================================================
# Visualization of collocation points: interior, IC, BC collocation points
# ============================================================

# ------------------------------------------------------------
# 1) Interior collocation points (for PDE residual)
# ------------------------------------------------------------

# Create 1D grids for space (x) and time (t).
# Here we choose 25 points in x ∈ [-1, 1] and 25 points in t ∈ [0, 1].
# -> resolution in each dimension is a design choice (DL engineer can tune this).
x_values = torch.linspace(-1, 1, 25).view(-1, 1)  # shape: (25, 1), spatial grid
t_values = torch.linspace(0, 1, 25).view(-1, 1)   # shape: (25, 1), temporal grid

# Create a 2D meshgrid of (x, t) pairs.
# x_values.squeeze(): shape (25,) / t_values.squeeze(): shape (25,)
# indexing='xy' means:
#   - first output (x_collocation) corresponds to x-axis (horizontal)
#   - second output (t_collocation) corresponds to y-axis (vertical)
# The resulting x_collocation, t_collocation have shape (25, 25).
x_collocation, t_collocation = torch.meshgrid(
    x_values.squeeze(),
    t_values.squeeze(),
    indexing='xy'
)

# Reshape the meshgrid into a list of collocation points.
# (25, 25) -> (25*25, 1) = (625, 1)
# Each row of (x_collocation, t_collocation) now corresponds to one (x_i, t_i) point
# inside the spatio-temporal domain Ω = [-1,1] × [0,1].
x_collocation = x_collocation.reshape(-1, 1)
t_collocation = t_collocation.reshape(-1, 1)

# ------------------------------------------------------------
# 2) Boundary points (for Dirichlet/Neumann boundary conditions)
# ------------------------------------------------------------

# For Burgers / heat equation in 1D, typical boundaries are at x = -1 and x = 1.
# We will generate boundary points for all time values t ∈ [0,1]
# on both the left and right boundary.

# Left boundary: x = -1 for all t_values
# torch.full_like(t_values, -1) creates a tensor with the same shape as t_values
# filled with -1 → x = -1 at each time point.
x_boundary_left = torch.full_like(t_values, -1)   # shape: (25,1)

# Right boundary: x = 1 for all t_values
x_boundary_right = torch.full_like(t_values, 1)   # shape: (25,1)

# Time coordinates for boundary points are exactly t_values.
# So boundary points are:
#   Left:  (x=-1, t=t_values[k])
#   Right: (x= 1, t=t_values[k])
t_boundary_points = t_values                      # shape: (25,1)

# ------------------------------------------------------------
# 3) Initial condition points (for u(x, t=0))
# ------------------------------------------------------------

# Initial condition is usually given at t = 0 for all x in [-1,1].
# So we set t = 0 for all spatial points.
t_initial_condition = torch.zeros_like(x_values)  # shape: (25,1), all zeros -> t = 0

# For the IC, x just spans over the spatial grid.
x_initial_condition = x_values                    # shape: (25,1), x ∈ [-1,1]

# At these points (x_initial_condition, t_initial_condition),
# we enforce u(x, 0) = u0(x).

# ------------------------------------------------------------
# 4) Convert to NumPy for plotting (Matplotlib expects NumPy)
# ------------------------------------------------------------

# Interior collocation points (PDE residual)
x_collocation_np = x_collocation.numpy()
t_collocation_np = t_collocation.numpy()

# Initial condition points
x_initial_condition_np = x_initial_condition.numpy()
t_initial_condition_np = t_initial_condition.numpy()

# Boundary points
x_boundary_left_np = x_boundary_left.numpy()
x_boundary_right_np = x_boundary_right.numpy()
t_boundary_points_np = t_boundary_points.numpy()

# ------------------------------------------------------------
# 5) Visualize the geometry and different sets of points
# ------------------------------------------------------------

plt.figure(figsize=(6, 4))

# Interior (domain) points: where PDE residual is enforced
plt.scatter(
    x_collocation_np, t_collocation_np,
    label='Domain Points (PDE residual)',
    color='blue', s=10
)

# Initial condition points: t = 0 line
plt.scatter(
    x_initial_condition_np, t_initial_condition_np,
    label='Initial Condition Points (t = 0)',
    color='green', s=30
)

# Left boundary: x = -1 over all t
plt.scatter(
    x_boundary_left_np, t_boundary_points_np,
    label='Left Boundary Points (x = -1)',
    color='red', s=30
)

# Right boundary: x = 1 over all t
plt.scatter(
    x_boundary_right_np, t_boundary_points_np,
    label='Right Boundary Points (x = 1)',
    color='orange', s=30
)

# Label axes: x-axis is space, y-axis is time
plt.xlabel('x')
plt.ylabel('t')
plt.title('Collocation, Initial, and Boundary Points in (x, t) Domain')

# Put the legend outside the plot to avoid cluttering the figure area
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.grid(True)
plt.show()


In [ ]:
### DeepXDE utilized

!pip install deepxde

import deepxde as dde
dde.backend.set_default_backend("pytorch")
from deepxde.backend import torch, backend_name

import numpy as np
import matplotlib.pyplot as plt

'''
"# DeepXDE will internally create tf.keras layers with these specs."
If there is any issue with torch.sin or other torch.###,
just replace it with tf.sin or tf.###.
'''

# # For reproducibility: fix all random seeds used by DeepXDE / TensorFlow backend
# dde.config.set_random_seed(1234)


In [ ]:
import deepxde as dde
import numpy as np
import torch


# PDE: 1D viscous Burgers' equation u_t + u u_x - ν u_xx = 0
def pde(x, y):
    # First derivatives: y_x, y_t
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    # Second derivative in space: y_xx
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t + y * dy_x - 0.01 / np.pi * dy_xx


# ---- Boundary condition helpers ----
def bc_fn(x, on_boundary):
    """Return True on spatial boundary x = -1 or x = 1."""
    if not on_boundary:
        return False
    return np.isclose(x[0], -1) or np.isclose(x[0], 1)

def bc_val(x):
    """Dirichlet boundary value u(t, ±1) = 0."""
    return 0

# ---- Initial condition helpers ----
def ic_fn(x, on_initial):
    """Return True on initial line t = 0 for all x."""
    return on_initial and dde.utils.isclose(x[1], 0)

def ic_val(x):
    """Initial profile u(0, x) = -sin(π x)."""
    return -np.sin(np.pi * x[:, 0:1])

# ---- Geometry in (x, t) ----
geom = dde.geometry.Interval(-1, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# ---- BC and IC objects ----
bc = dde.icbc.DirichletBC(geomtime, bc_val, bc_fn)
ic = dde.icbc.IC(geomtime, ic_val, ic_fn)

# ---- Data and model definition ----
data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc, ic],
    num_domain=2540,   # interior collocation points
    num_boundary=80,   # boundary points
    num_initial=160,   # initial-condition points
)

layer_size = [2] + [20] * 3 + [1]   # (x,t) → 3 hidden layers → u
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)
model = dde.Model(data, net)


# Custom metric: always evaluate L2 error on external reference grid
def l2_rel_err_ext(_, y_pred):
    """L2 relative error on (X_ref, Y_ref) instead of internal DeepXDE test set."""
    y_test_pred = model.predict(X_ref)
    return dde.metrics.l2_relative_error(Y_ref, y_test_pred)


# ---- Compile and train with Adam ----
model.compile(
    "adam",
    lr=1e-3,
    metrics=[l2_rel_err_ext])

losshistory, train_state = model.train(iterations=15000)


# ---- Final evaluation on external test data ----
y_pred = model.predict(X_ref)
final_l2_error = dde.metrics.l2_relative_error(Y_ref, y_pred)
f = model.predict(X_ref, operator=pde)  # PDE residual at test points

print("Final L2 relative error (on external test data): {:.2e}".format(final_l2_error))

# ---- Generate grid and plot results ----
t = np.linspace(0, 1, 100)
x = np.linspace(-1, 1, 256)
xx, tt = np.meshgrid(x, t)
X = np.vstack((xx.ravel(), tt.ravel())).T

y_pred = model.predict(X)
y_pred_reshaped = y_pred.reshape(len(t), len(x))

plt.figure(figsize=(16, 6))

# Reference solution
plt.subplot(1, 2, 1)
plt.contourf(xx, tt, exact, levels=50, cmap="jet")
plt.colorbar(label="u")
plt.title("Reference Solution of Burgers' Equation")
plt.xlabel("x")
plt.ylabel("t")

# PINN solution
plt.subplot(1, 2, 2)
plt.contourf(xx, tt, y_pred_reshaped, levels=50, cmap="jet")
plt.colorbar(label="u")
plt.title("Solution of Burgers' Equation using PINNs (CPU)")
plt.xlabel("x")
plt.ylabel("t")

plt.tight_layout()
plt.show()



In [ ]:
# Time instances for line plots
time_slices = [0.0, 0.25, 0.50, 0.75, 1.0]

# Determine rows and columns for subplots
num_plots = len(time_slices)
cols = 2  # Two columns for better arrangement
rows = (num_plots + cols - 1) // cols  # Ceiling division for rows

# Create the subplot grid
fig, axs = plt.subplots(rows, cols, figsize=(12, 5 * rows))  # Adjust height based on rows
axs = axs.flatten()  # Flatten for easy indexing

# Loop through time slices and plot solutions
for i, t_val in enumerate(time_slices):
    # Find the index corresponding to the closest time value
    idx = np.argmin(np.abs(t - t_val))

    # Extract solutions at the specific time slice
    y_analytical = exact[idx, :]
    y_pinn = y_pred_reshaped[idx, :]

    # Plot analytical and predicted solutions
    axs[i].plot(x, y_analytical, 'r-', label='Analytical Solution')
    axs[i].plot(x, y_pinn, 'b--', label='PINN Solution')
    axs[i].set_xlabel('x')
    axs[i].set_ylabel('u')
    axs[i].set_title(f'Solution at t = {t_val}')
    axs[i].legend()

# Remove any empty subplots if the grid has extra spaces
for j in range(num_plots, len(axs)):
    fig.delaxes(axs[j])

# Adjust layout for neatness
plt.tight_layout()
plt.show()
